## **API WHEATHER**

 Ofrece una variedad de datos y funcionalidades para obtener información sobre el clima y las condiciones meteorológicas de cualquier lugar.

In [ ]:
import requests  # Biblioteca para realizar solicitudes HTTP
import matplotlib.pyplot as plt

# Función para obtener datos del clima desde WeatherAPI
def obtener_clima(ciudad, api_key_weather): # conecta con la API de Weather Api para opbtener los datos climáticos de una ciudad específica
    url_weather = f"http://api.weatherapi.com/v1/current.json?key={api_key_weather}&q={ciudad}" #Se construye URL dinámica, current.json para datos actuales, la clave de API (api_key_weather) y el parametro de busqueda (ciudad)
    try: #solicitamos datos a la API
        respuesta = requests.get(url_weather, timeout=10) #Hace una solicitud GET a la URL, limita el tiempo de espera 10seg
        respuesta.raise_for_status() # Lanza una excepción si la respuesta indica un error
        datos = respuesta.json() #Facilita el acceso convierte la respuesta JSON en un diccionario Python
        # Extraer datos relevantes
        clima = { #se construye diccionario clima con los datos relevantes como: Ciudad, país, temp °C, humedad%, velocidad viento km/h.
            "ciudad": datos.get("location", {}).get("name"), #usando get() se extraen valores especificos
            "pais": datos.get("location", {}).get("country"),
            "temperatura_c": datos.get("current", {}).get("temp_c"),
            "humedad": datos.get("current", {}).get("humidity"),
            "viento_kph": datos.get("current", {}).get("wind_kph"),
            "descripcion": datos.get("current", {}).get("condition", {}).get("text") #descripcion textual de las condiciones actuales
        }
        return clima
    except requests.exceptions.RequestException as e: #captura cualquier excepción generada durante la solicitud (error de conexion, tiempo de espera)
        print(f"Error al obtener el clima de {ciudad}: {e}")
        return None

# Ejemplo de uso
api_key_weather = "e1d44396e7d44430887215118241812"  # Reemplaza con tu clave de API
ciudad = "Lima" #se especifica la ciudad
clima = obtener_clima(ciudad, api_key_weather) #se llama la función y se guarda el resultado en clima.

if clima: #si clima no es None imprime los datos climáticos.
    print("Datos del clima:")
    print(f"Ciudad: {clima['ciudad']}, País: {clima['pais']}")
    print(f"Temperatura: {clima['temperatura_c']}°C")
    print(f"Humedad: {clima['humedad']}%")
    print(f"Viento: {clima['viento_kph']} km/h")
    print(f"Condiciones: {clima['descripcion']}")



Datos del clima:
Ciudad: Lima, País: Peru
Temperatura: 20.1°C
Humedad: 81%
Viento: 14.0 km/h
Condiciones: Partly Cloudy


## **API KEY IQAIR**
Esta API Ofrece información detallada sobre la calidad del aire (AQI) y contaminantes en diferentes ubicaciones del mundo.



In [ ]:
import requests

# Función para obtener datos de calidad del aire desde IQAir usando el nombre de una ciudad
def obtener_calidad_aire_ciudad(ciudad, estado, pais, api_key_iqair): #Obtiene datos de calidad del aire de IQAir para una ciudad específica.
    url_iqair = f"https://api.airvisual.com/v2/city?city={ciudad}&state={estado}&country={pais}&key={api_key_iqair}"
    try:
        respuesta = requests.get(url_iqair, timeout=10)  #solicita GET a la URL, limita el tiempo de espera 10seg
        respuesta.raise_for_status() # Lanza una excepción si la respuesta indica un error
        datos = respuesta.json() #Facilita el acceso convierte la respuesta JSON en un diccionario Python
        # Extraer datos relevantes
        calidad_aire = {
            "ciudad": datos.get("data", {}).get("city"),
            "estado": datos.get("data", {}).get("state"),
            "pais": datos.get("data", {}).get("country"),
            "aqi_us": datos.get("data", {}).get("current", {}).get("pollution", {}).get("aqius"),
            "principal_contaminante": datos.get("data", {}).get("current", {}).get("pollution", {}).get("mainus"),
            "temperatura_c": datos.get("data", {}).get("current", {}).get("weather", {}).get("tp"),
            "humedad": datos.get("data", {}).get("current", {}).get("weather", {}).get("hu"),
            "velocidad_viento_m/s": datos.get("data", {}).get("current", {}).get("weather", {}).get("ws")
        }
        return calidad_aire
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener la calidad del aire para {ciudad}, {estado}, {pais}: {e}")
        return None

# Ejemplo de uso
api_key_iqair = "b79f3699-982d-4eb7-ac33-f15226399808"  # Reemplaza con tu clave de API
ciudad = "Lima"  # Ejemplo: ciudad
estado = "Lima"  # Ejemplo: estado
pais = "Peru"  # Ejemplo: país

calidad_aire = obtener_calidad_aire_ciudad(ciudad, estado, pais, api_key_iqair)

if calidad_aire:
    print("Datos de calidad del aire:")
    print(f"Ciudad: {calidad_aire['ciudad']}, Estado: {calidad_aire['estado']}, País: {calidad_aire['pais']}")
    print(f"AQI (US): {calidad_aire['aqi_us']}")
    print(f"Principal contaminante: {calidad_aire['principal_contaminante']}")
    print(f"Temperatura: {calidad_aire['temperatura_c']}°C")
    print(f"Humedad: {calidad_aire['humedad']}%")
    print(f"Velocidad del viento: {calidad_aire['velocidad_viento_m/s']} m/s")


Datos de calidad del aire:
Ciudad: Lima, Estado: Lima, País: Peru
AQI (US): 54
Principal contaminante: p2
Temperatura: 21°C
Humedad: 78%
Velocidad del viento: 6.69 m/s


## **Integración de APIS**

In [ ]:
import pandas as pd

# Función para comparar ciudades según su clima y calidad del aire
def comparar_ciudades(ciudades, api_key_iqair, api_key_weather):
    datos_aire = []  # Lista para almacenar datos de calidad del aire
    datos_clima = []  # Lista para almacenar datos del clima

    # Obtener datos de calidad del aire para cada ciudad
    for ciudad_info in ciudades:
        # Llama a la función obtener_calidad_aire para cada ciudad en la lista
        calidad_aire = obtener_calidad_aire(
            ciudad_info['ciudad'],  # Nombre de la ciudad
            ciudad_info['estado'],  # Nombre del estado
            ciudad_info['pais'],    # Nombre del país
            api_key_iqair           # Clave de acceso a la API IQAir
        )
        if calidad_aire:  # Verifica que los datos sean válidos
            datos_aire.append(calidad_aire)  # Agrega los datos a la lista

    # Obtener datos de clima para cada ciudad
    for ciudad_info in ciudades:
        # Llama a la función obtener_clima para cada ciudad en la lista
        clima = obtener_clima(
            ciudad_info['ciudad'],  # Nombre de la ciudad
            api_key_weather         # Clave de acceso a la API WeatherAPI
        )
        if clima:  # Verifica que los datos sean válidos
            datos_clima.append(clima)  # Agrega los datos a la lista

    # Crear un DataFrame de Pandas con los datos de calidad del aire
    df_aire = pd.DataFrame(datos_aire)  # Convierte la lista en un DataFrame

    # Crear un DataFrame de Pandas con los datos del clima
    df_clima = pd.DataFrame(datos_clima)  # Convierte la lista en un DataFrame

    # Combinar los DataFrames por el nombre de la ciudad
    comparacion_df = pd.merge(
        df_aire,      # Primer DataFrame: calidad del aire
        df_clima,     # Segundo DataFrame: datos del clima
        on="ciudad",  # Columna clave para combinar los DataFrames
        how="inner"   # Tipo de combinación: solo coincidencias en ambas tablas
    )

    return comparacion_df  # Devuelve el DataFrame combinado

# Definir las ciudades que deseas analizar
ciudades = [
    {"ciudad": "Piura", "estado": "Piura", "pais": "Peru"},         # Información de Piura
    {"ciudad": "Lima", "estado": "Lima", "pais": "Peru"},           # Información de Lima
    {"ciudad": "Arequipa", "estado": "Arequipa", "pais": "Peru"},   # Información de Arequipa
    {"ciudad": "Trujillo", "estado": "La Libertad", "pais": "Peru"} # Información de Trujillo
]

# Claves API
api_key_iqair = "88748538-fe54-4cf4-8ec8-6e0934fd13d0"  # Clave para IQAir
api_key_weather = "e1d44396e7d44430887215118241812"      # Clave para WeatherAPI

# Comparar las ciudades y almacenar el resultado en un DataFrame
comparacion = comparar_ciudades(ciudades, api_key_iqair, api_key_weather)

# Imprimir el DataFrame con los resultados
print(comparacion)


     ciudad       estado pais_x  aqi_us principal_contaminante  \
0     Piura        Piura   Peru      47                     p2   
1      Lima         Lima   Peru      54                     p2   
2  Arequipa     Arequipa   Peru      58                     p2   
3  Trujillo  La Libertad   Peru      37                     p2   

   temperatura_c_x  humedad_x  velocidad_viento_m/s pais_y  temperatura_c_y  \
0               22         66                  8.54   Peru             22.6   
1               21         78                  6.69   Peru             20.1   
2               16         63                  0.51   Peru             10.9   
3               18         82                  5.66   Peru             18.7   

   humedad_y  viento_kph    descripcion  
0         67        30.2  Partly Cloudy  
1         81        14.0  Partly Cloudy  
2         68         4.3          Clear  
3         86        10.1          Clear  


## **Comparación con solo los datos relevantes**

In [ ]:

# Mostrar la tabla de comparación con solo los datos relevantes
print("\nComparación de ciudades:")  # Imprime un encabezado para la tabla de comparación
# Muestra solo las columnas seleccionadas del DataFrame `comparacion`
print(comparacion[['ciudad', 'aqi_us', 'temperatura_c_x', 'humedad_x', 'velocidad_viento_m/s']])

# Determinar la mejor ciudad
# `idxmin()` encuentra el índice de la fila con el valor mínimo en la columna 'aqi_us' (mejor calidad del aire)
mejor_ciudad = comparacion.loc[comparacion['aqi_us'].idxmin()]

# Imprime un mensaje indicando que se muestra la mejor ciudad
mejor_ciudad = comparacion.loc[comparacion['aqi_us'].idxmin()]
print("\nLa mejor ciudad según los datos obtenidos es:")
print(f"Ciudad: {mejor_ciudad['ciudad']}")
print(f"Calidad del aire (AQI US): {mejor_ciudad['aqi_us']}")
print(f"Temperatura: {mejor_ciudad['temperatura_c_x']}°C")
print(f"Humedad: {mejor_ciudad['humedad_x']}%")
print(f"Velocidad del viento: {mejor_ciudad['velocidad_viento_m/s']} m/s")

# **Índice de habitabilidad basado en las métricas obtenidas y clasificar las ciudades en categorías**

In [ ]:
import pandas as pd  # Importar la librería pandas para manejar datos en formato de tablas (dataframes)

# Agregar una nueva columna al DataFrame para calcular el índice de habitabilidad
def calcular_indice_habitabilidad(df):
    # Normalizar las métricas para que todas tengan valores comparables

    # Calcular un índice para la calidad del aire basado en el AQI (Air Quality Index). Invertimos el AQI para que valores más bajos (mejor calidad) tengan un puntaje más alto.
    df['indice_calidad_aire'] = 100 - df['aqi_us']  # Invertir AQI, valores más bajos son mejores

    # Calcular el índice de temperatura: Si está en el rango de 15-30°C, se considera ideal, de lo contrario se asigna un valor de 50.
    df['indice_temperatura'] = df['temperatura_c_x'].apply(lambda x: 100 if 15 <= x <= 30 else 50)

    # Calcular el índice de humedad: Si está en el rango de 30%-60%, se considera ideal, de lo contrario se asigna un valor de 50.
    df['indice_humedad'] = df['humedad_x'].apply(lambda x: 100 if 30 <= x <= 60 else 50)

    # Calcular el índice de viento: Si la velocidad del viento es mayor o igual a 2 m/s, se considera ideal, de lo contrario se asigna un valor de 50.
    df['indice_viento'] = df['velocidad_viento_m/s'].apply(lambda x: 100 if x >= 2 else 50)

    # Promediar los índices para obtener un índice general de habitabilidad
    # El índice de habitabilidad es el promedio de los índices de calidad del aire, temperatura, humedad y viento.
    df['indice_habitabilidad'] = (
        df['indice_calidad_aire'] +  # Índice de calidad del aire
        df['indice_temperatura'] +  # Índice de temperatura
        df['indice_humedad'] +      # Índice de humedad
        df['indice_viento']         # Índice de viento
    ) / 4  # Promedio de los 4 índices

    # Clasificar según umbrales
    # Función que clasifica la habitabilidad en categorías de "Alta Habitabilidad", "Moderada Habitabilidad" y "Baja Habitabilidad"
    def clasificar_habitabilidad(indice):
        if indice >= 75:  # Si el índice es mayor o igual a 75, la ciudad tiene alta habitabilidad
            return "Alta Habitabilidad"
        elif indice >= 50:  # Si el índice es mayor o igual a 50, pero menor a 75, la ciudad tiene moderada habitabilidad
            return "Moderada Habitabilidad"
        else:  # Si el índice es menor a 50, la ciudad tiene baja habitabilidad
            return "Baja Habitabilidad"

    # Aplicamos la función de clasificación a la columna 'indice_habitabilidad' para crear una nueva columna 'categoria_habitabilidad'
    df['categoria_habitabilidad'] = df['indice_habitabilidad'].apply(clasificar_habitabilidad)

    return df  # Devolvemos el DataFrame con las nuevas columnas y categorías

# Aplicar la función al DataFrame de comparación
comparacion = calcular_indice_habitabilidad(comparacion)  # Calculamos el índice de habitabilidad para cada ciudad

# Mostrar los resultados con las columnas relevantes
print("\nClasificación de habitabilidad de las ciudades:")
# Mostramos la ciudad, el índice de habitabilidad y la categoría de habitabilidad
print(comparacion[['ciudad', 'indice_habitabilidad', 'categoria_habitabilidad']])

# Identificar la ciudad con el índice de habitabilidad más alto (la mejor ciudad)
ciudad_ideal = comparacion.loc[comparacion['indice_habitabilidad'].idxmax()]  # Encontramos la ciudad con el índice máximo
print("\nLa ciudad con mayor habitabilidad es:")  # Mostramos el resultado
print(f"Ciudad: {ciudad_ideal['ciudad']}")
print(f"Índice de Habitabilidad: {ciudad_ideal['indice_habitabilidad']:.2f}")  # Mostramos el índice con dos decimales
print(f"Categoría: {ciudad_ideal['categoria_habitabilidad']}")  # Mostramos la categoría de la ciudad ideal


Este proceso te permitirá comparar y clasificar las ciudades según su habitabilidad, priorizando la salud respiratoria en función de los datos climáticos y de calidad del aire obtenidos.

## **Evaluación de Riesgos para la salud**

In [ ]:
import pandas as pd  # Importa la biblioteca pandas para manipular y analizar datos tabulares

# Función para evaluar riesgos en base al AQI y condiciones climáticas extremas
def evaluar_riesgos_salud(df):
    """
    Esta función evalúa los riesgos para la salud de cada ciudad en función de la calidad del aire (AQI),
    la temperatura y la humedad, clasificando el riesgo como alto, moderado o bajo.
    """

    # Crear una nueva columna para evaluar el riesgo de calidad del aire según el AQI
    def riesgo_calidad_aire(aqi):
        """
        Evalúa el riesgo relacionado con la calidad del aire basándose en el AQI.
        El AQI más alto indica un mayor riesgo para la salud.
        """
        if aqi > 150:
            return "Riesgo alto de problemas respiratorios (población general)"
        elif aqi > 100:
            return "Riesgo moderado para personas sensibles"
        else:
            return "Riesgo bajo"

    # Crear una nueva columna para evaluar el riesgo por temperatura extrema
    def riesgo_temperatura(temperatura):
        """
        Evalúa el riesgo relacionado con temperaturas extremas.
        Las temperaturas altas o bajas extremas representan riesgos para la salud.
        """
        if temperatura > 35:
            return "Riesgo alto debido a calor extremo"
        elif temperatura < 0:
            return "Riesgo alto debido a frío extremo"
        else:
            return "Riesgo bajo"

    # Crear una nueva columna para evaluar el riesgo por humedad extrema
    def riesgo_humedad(humedad):
        """
        Evalúa el riesgo relacionado con niveles de humedad extremas.
        La alta o baja humedad puede afectar la respiración y la salud en general.
        """
        if humedad > 70:
            return "Riesgo alto por humedad excesiva"
        elif humedad < 20:
            return "Riesgo alto por baja humedad"
        else:
            return "Riesgo bajo"

    # Evaluar los riesgos para cada ciudad, asignando valores a las columnas
    df['riesgo_aire'] = df['aqi_us'].apply(riesgo_calidad_aire)  # Se aplica la función de riesgo por aire al AQI de cada ciudad
    df['riesgo_temperatura'] = df['temperatura_c_x'].apply(riesgo_temperatura)  # Se aplica la función de riesgo por temperatura a cada ciudad
    df['riesgo_humedad'] = df['humedad_x'].apply(riesgo_humedad)  # Se aplica la función de riesgo por humedad a cada ciudad

    # Función para calcular el riesgo global de la salud combinando los riesgos de aire, temperatura y humedad
    def riesgo_global(row):
        """
        Combina los riesgos por calidad del aire, temperatura y humedad para determinar el riesgo global.
        Si cualquiera de los riesgos es alto, el riesgo global será alto.
        """
        riesgos = [row['riesgo_aire'], row['riesgo_temperatura'], row['riesgo_humedad']]  # Lista de todos los riesgos de la ciudad
        # Si cualquiera de los riesgos es alto, el riesgo global es alto. Si es moderado, el riesgo global es moderado.
        if "Riesgo alto" in riesgos:
            return "Riesgo alto para la salud"
        elif "Riesgo moderado" in riesgos:
            return "Riesgo moderado para la salud"
        else:
            return "Riesgo bajo para la salud"

    # Crear una columna para el riesgo global
    df['riesgo_global'] = df.apply(riesgo_global, axis=1)  # Aplica la función de riesgo global a cada fila del DataFrame

    return df  # Devuelve el DataFrame con las nuevas columnas de riesgos

# Aplicar la función a los datos obtenidos
comparacion = evaluar_riesgos_salud(comparacion)  # Llama a la función y le pasa el DataFrame 'comparacion' con datos de ciudades

# Mostrar los resultados con las evaluaciones de riesgos
print("\nEvaluación de riesgos para la salud en las ciudades:")
# Muestra una tabla con la ciudad y los distintos riesgos
print(comparacion[['ciudad', 'riesgo_aire', 'riesgo_temperatura', 'riesgo_humedad', 'riesgo_global']])

# Identificar la ciudad con el menor riesgo para la salud
ciudad_sin_riesgo = comparacion.loc[comparacion['riesgo_global'] == "Riesgo bajo para la salud"]  # Filtra las ciudades con riesgo bajo
print("\nCiudades con el menor riesgo para la salud:")
# Itera a través de las ciudades con el menor riesgo y muestra los resultados
for index, row in ciudad_sin_riesgo.iterrows():
    print(f"Ciudad: {row['ciudad']}")  # Muestra el nombre de la ciudad
    print(f"Riesgo de aire: {row['riesgo_aire']}")  # Muestra el riesgo asociado al aire
    print(f"Riesgo de temperatura: {row['riesgo_temperatura']}")  # Muestra el riesgo asociado a la temperatura
    print(f"Riesgo de humedad: {row['riesgo_humedad']}")  # Muestra el riesgo asociado a la humedad
    print(f"Riesgo global: {row['riesgo_global']}")  # Muestra el riesgo global para la salud de la ciudad


Este enfoque permite evaluar los riesgos para la salud respiratoria en diversas ciudades del Perú en función de la calidad del aire y las condiciones climáticas extremas.